# Variance Risk Premium Analysis

This notebook analyzes the Variance Risk Premium (VRP) - the difference between implied and realized volatility.

**Contents:**
1. VRP Overview
2. Calculate IV and RV
3. VRP Time Series
4. Statistical Properties
5. Trading Signal Analysis

In [ ]:
# Standard imports
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import date, timedelta
from scipy import stats

# Volsurf imports
from volsurf.database.connection import get_connection
from volsurf.database.schema import init_schema
from volsurf.analytics import (
    SurfaceMetrics,
    RealizedVolCalculator,
    VRPCalculator
)

# Initialize
init_schema()
conn = get_connection()

SYMBOL = 'SPY'

## 1. VRP Overview

**Variance Risk Premium (VRP)** = Implied Volatility - Realized Volatility

- **Positive VRP**: Options are "expensive" - IV > what the underlying actually moves
- **Negative VRP**: Options are "cheap" - IV < actual realized moves

VRP is typically positive, representing the premium option sellers collect for bearing volatility risk.

In [ ]:
# Get available date range
metrics = SurfaceMetrics()
available_dates = metrics.get_available_dates(SYMBOL)

if not available_dates:
    print("No data available. Run ingestion and fitting first.")
else:
    print(f"Date range: {available_dates[0]} to {available_dates[-1]}")
    print(f"Trading days: {len(available_dates)}")

## 2. Calculate Implied and Realized Volatility

In [ ]:
# Get ATM implied volatility (30-day)
start_date = available_dates[0]
end_date = available_dates[-1]

iv_df = metrics.get_atm_vol_timeseries(SYMBOL, start_date, end_date, tte_target_days=30)

print(f"IV data points: {len(iv_df)}")
if not iv_df.empty:
    print(f"\nIV Statistics (30-day ATM):")
    print(f"  Mean: {iv_df['atm_vol'].mean():.2%}")
    print(f"  Std:  {iv_df['atm_vol'].std():.2%}")
    print(f"  Min:  {iv_df['atm_vol'].min():.2%}")
    print(f"  Max:  {iv_df['atm_vol'].max():.2%}")

In [ ]:
# Get realized volatility (21-day, closest to 30 calendar days)
rv_calc = RealizedVolCalculator()
rv_df = rv_calc.get_vol_timeseries(SYMBOL, start_date, end_date, metric='rv_21d')

print(f"RV data points: {len(rv_df)}")
if not rv_df.empty:
    print(f"\nRV Statistics (21-day):")
    print(f"  Mean: {rv_df['realized_vol'].mean():.2%}")
    print(f"  Std:  {rv_df['realized_vol'].std():.2%}")
    print(f"  Min:  {rv_df['realized_vol'].min():.2%}")
    print(f"  Max:  {rv_df['realized_vol'].max():.2%}")

In [ ]:
# Plot IV vs RV
fig = go.Figure()

if not iv_df.empty:
    fig.add_trace(go.Scatter(
        x=iv_df['date'], y=iv_df['atm_vol'] * 100,
        mode='lines', name='30-Day IV',
        line=dict(color='blue')
    ))

if not rv_df.empty:
    fig.add_trace(go.Scatter(
        x=rv_df['date'], y=rv_df['realized_vol'] * 100,
        mode='lines', name='21-Day RV',
        line=dict(color='orange')
    ))

fig.update_layout(
    title=f'{SYMBOL} Implied vs Realized Volatility',
    xaxis_title='Date',
    yaxis_title='Volatility (%)',
    hovermode='x unified',
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01)
)

fig.show()

## 3. VRP Time Series

In [ ]:
# Get VRP time series
vrp_calc = VRPCalculator()
vrp_df = vrp_calc.get_vrp_timeseries(SYMBOL, start_date, end_date)

if vrp_df.empty:
    print("No VRP data. Computing manually...")
    # Merge IV and RV
    if not iv_df.empty and not rv_df.empty:
        merged = pd.merge(
            iv_df[['date', 'atm_vol']],
            rv_df[['date', 'realized_vol']],
            on='date',
            how='inner'
        )
        merged['vrp_30d'] = merged['atm_vol'] - merged['realized_vol']
        vrp_df = merged
        print(f"Computed VRP for {len(vrp_df)} dates")

if not vrp_df.empty:
    print(f"\nVRP Statistics:")
    print(f"  Mean: {vrp_df['vrp_30d'].mean():.2%}")
    print(f"  Std:  {vrp_df['vrp_30d'].std():.2%}")
    print(f"  % Positive: {(vrp_df['vrp_30d'] > 0).mean():.1%}")

In [ ]:
# Plot VRP time series
if not vrp_df.empty:
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=vrp_df['date'],
        y=vrp_df['vrp_30d'] * 100,
        mode='lines',
        name='VRP (30d)',
        fill='tozeroy',
        fillcolor='rgba(0,100,80,0.2)',
        line=dict(color='rgb(0,100,80)')
    ))
    
    fig.add_hline(y=0, line_dash='dash', line_color='gray')
    fig.add_hline(y=vrp_df['vrp_30d'].mean()*100, line_dash='dot', line_color='blue',
                  annotation_text=f"Mean: {vrp_df['vrp_30d'].mean():.2%}")
    
    fig.update_layout(
        title=f'{SYMBOL} Variance Risk Premium (IV - RV)',
        xaxis_title='Date',
        yaxis_title='VRP (%)',
        hovermode='x unified'
    )
    
    fig.show()

## 4. Statistical Properties of VRP

In [ ]:
if not vrp_df.empty:
    vrp_values = vrp_df['vrp_30d'].dropna()
    
    # Distribution plot
    fig = make_subplots(rows=1, cols=2, subplot_titles=['VRP Distribution', 'VRP Q-Q Plot'])
    
    # Histogram
    fig.add_trace(
        go.Histogram(x=vrp_values * 100, nbinsx=30, name='VRP'),
        row=1, col=1
    )
    
    # Q-Q plot
    qq = stats.probplot(vrp_values, dist='norm')
    fig.add_trace(
        go.Scatter(x=qq[0][0], y=qq[0][1]*100, mode='markers', name='Data'),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(x=qq[0][0], y=qq[1][0]*qq[0][0]*100 + qq[1][1]*100, 
                   mode='lines', name='Normal', line=dict(color='red')),
        row=1, col=2
    )
    
    fig.update_layout(height=400, showlegend=False)
    fig.update_xaxes(title_text='VRP (%)', row=1, col=1)
    fig.update_xaxes(title_text='Theoretical Quantiles', row=1, col=2)
    fig.update_yaxes(title_text='VRP (%)', row=1, col=2)
    
    fig.show()
    
    # Statistical tests
    print("\nStatistical Tests:")
    print(f"  Skewness: {stats.skew(vrp_values):.3f}")
    print(f"  Kurtosis: {stats.kurtosis(vrp_values):.3f}")
    
    # Test if mean is significantly different from zero
    t_stat, p_value = stats.ttest_1samp(vrp_values, 0)
    print(f"\nT-test (H0: mean=0):")
    print(f"  t-statistic: {t_stat:.3f}")
    print(f"  p-value: {p_value:.4f}")
    print(f"  Mean significantly different from zero: {p_value < 0.05}")

In [ ]:
# VRP percentiles and Z-score
if not vrp_df.empty:
    vrp_values = vrp_df['vrp_30d'].dropna()
    current_vrp = vrp_values.iloc[-1]
    
    percentile = stats.percentileofscore(vrp_values, current_vrp)
    zscore = (current_vrp - vrp_values.mean()) / vrp_values.std()
    
    print(f"\nCurrent VRP Analysis:")
    print(f"  Current VRP: {current_vrp:.2%}")
    print(f"  Percentile: {percentile:.1f}%")
    print(f"  Z-Score: {zscore:.2f}")
    
    if zscore > 1.5:
        print("  -> VRP is elevated (short vol opportunity?)")
    elif zscore < -1.5:
        print("  -> VRP is depressed (long vol opportunity?)")
    else:
        print("  -> VRP is within normal range")

## 5. Trading Signal Analysis

Explore potential trading signals based on VRP.

In [ ]:
# VRP regime analysis
if not vrp_df.empty and len(vrp_df) > 20:
    vrp_df_analysis = vrp_df.copy()
    
    # Calculate rolling statistics
    window = min(20, len(vrp_df) - 1)
    vrp_df_analysis['vrp_ma'] = vrp_df_analysis['vrp_30d'].rolling(window).mean()
    vrp_df_analysis['vrp_std'] = vrp_df_analysis['vrp_30d'].rolling(window).std()
    vrp_df_analysis['vrp_zscore'] = (
        (vrp_df_analysis['vrp_30d'] - vrp_df_analysis['vrp_ma']) / vrp_df_analysis['vrp_std']
    )
    
    # Define regimes
    vrp_df_analysis['regime'] = 'Normal'
    vrp_df_analysis.loc[vrp_df_analysis['vrp_zscore'] > 1.5, 'regime'] = 'High VRP'
    vrp_df_analysis.loc[vrp_df_analysis['vrp_zscore'] < -1.5, 'regime'] = 'Low VRP'
    
    print("Regime Distribution:")
    print(vrp_df_analysis['regime'].value_counts())

In [ ]:
# Plot VRP with regimes
if not vrp_df.empty and 'regime' in vrp_df_analysis.columns:
    fig = go.Figure()
    
    colors = {'Normal': 'gray', 'High VRP': 'green', 'Low VRP': 'red'}
    
    for regime in ['Normal', 'High VRP', 'Low VRP']:
        regime_df = vrp_df_analysis[vrp_df_analysis['regime'] == regime]
        if not regime_df.empty:
            fig.add_trace(go.Scatter(
                x=regime_df['date'],
                y=regime_df['vrp_30d'] * 100,
                mode='markers',
                name=regime,
                marker=dict(color=colors[regime], size=8)
            ))
    
    fig.add_hline(y=0, line_dash='dash', line_color='black')
    
    fig.update_layout(
        title=f'{SYMBOL} VRP Regimes',
        xaxis_title='Date',
        yaxis_title='VRP (%)',
        hovermode='x unified'
    )
    
    fig.show()

## Summary

Key findings from VRP analysis:

1. **VRP Level**: [Add observations]
2. **Distribution**: [Add observations]
3. **Current Signal**: [Add observations]